In [ ]:
import os
import glob
import datetime
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import pandas as pd
import scipy
from scipy.optimize import curve_fit

#from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment

import gsf_ims_fitness as fitness

import pickle

import gzip

import random

import seaborn as sns
sns.set()

#from sklearn.mixture import GaussianMixture
#from sklearn.mixture import BayesianGaussianMixture

%load_ext autoreload
%autoreload 2

%matplotlib inline

%autosave 0

sns.set_style("white")
sns.set_style("ticks", {'xtick.direction':'in', 'xtick.top':True, 'ytick.direction':'in', 'ytick.right':True})

plt.rcParams['axes.labelsize'] = 20
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

plt.rcParams['legend.fontsize'] = 14
plt.rcParams['legend.edgecolor'] = 'k'

In [ ]:
notebook_directory = os.getcwd()
notebook_directory

In [ ]:
experiment = 'output_file_label'

In [ ]:
data_directory = notebook_directory + "\\barcode_analysis"
os.chdir(data_directory)
os.getcwd()

In [ ]:
glob.glob("*reverse_barcode*.csv")

In [ ]:
reverse_barcode_map_file = glob.glob("*reverse_barcode*.csv")[0]
reverse_barcode_map_file

In [ ]:
rev_barcode_clusterID_frame = pd.read_csv(reverse_barcode_map_file, skipinitialspace=True)
rev_barcode_clusterID_frame[:5]

In [ ]:
reverse_barcode_center_file = glob.glob("*reverse_cluster*.csv")[0]
reverse_barcode_center_file

In [ ]:
rev_barcode_center_frame = pd.read_csv(reverse_barcode_center_file, skipinitialspace=True)
rev_barcode_center_frame.rename(columns={"time_point_1": "HiSeq_count"}, inplace=True)
rev_barcode_center_frame[:5]

In [ ]:
rev_barcode_center_frame.sort_values(by=['HiSeq_count'], ascending=False)[:5]

In [ ]:
rev_barcode_clusterID_dict = dict(zip(rev_barcode_clusterID_frame["Unique.reads"], rev_barcode_clusterID_frame["Cluster.ID"]))

for index, row in rev_barcode_center_frame.iterrows():
    rev_barcode_clusterID_dict[row["Center"]] = row["Cluster.ID"]

In [ ]:
forward_barcode_map_file = glob.glob("*forward_barcode*.csv")[0]
forward_barcode_map_file

In [ ]:
for_barcode_clusterID_frame = pd.read_csv(forward_barcode_map_file, skipinitialspace=True)
for_barcode_clusterID_frame[:5]

In [ ]:
forward_barcode_center_file = glob.glob("*forward_cluster*.csv")[0]
forward_barcode_center_file

In [ ]:
for_barcode_center_frame = pd.read_csv(forward_barcode_center_file, skipinitialspace=True)
for_barcode_center_frame.rename(columns={"time_point_1": "HiSeq_count"}, inplace=True)
for_barcode_center_frame[:5]

In [ ]:
for_barcode_center_frame.sort_values(by=['HiSeq_count'], ascending=False)[:5]

In [ ]:
for_barcode_clusterID_dict = dict(zip(for_barcode_clusterID_frame["Unique.reads"], for_barcode_clusterID_frame["Cluster.ID"]))

for index, row in for_barcode_center_frame.iterrows():
    for_barcode_clusterID_dict[row["Center"]] = row["Cluster.ID"]

In [ ]:
os.chdir(notebook_directory)
glob.glob("*pkl")

In [ ]:
os.chdir(notebook_directory)
pickle_file = experiment + '_BarSeqFitnessFrame.pkl'
print(pickle_file)

barcode_frame = pickle.load(open(pickle_file, 'rb'))

hiseq_count_frame = barcode_frame.barcode_frame

In [ ]:
len(hiseq_count_frame)

In [ ]:
hiseq_count_frame[:5]

In [ ]:
region = "empty_1"

wild_type_dict = {}
trim_dict = {}
wild_type_dict["empty_1"] = "CTAGCGCTGAGGTCTGCCTCGTGCAGCGAGTCAGTGAGCGAGGAAGCACCTCAGATAAAATATTTGCTCATGAGCCCGAAGTGGCGAGCCCGACAAAAAACCCCTCAAGACCCGTTTAGAGGCCCCAAGGGGTTATGCTAGTCTTCCCCATCGGTGAGCCCGGGCTGTCGGCGT"
trim_dict["empty_1"] = 25
wild_type_dict["empty_2"] = "CGGTGGCCCGGGCGGCCGCACGATGCGTCCGGCGTAGAGGATCTGCTCATGTTTGACAGCTTATCATCGATGCATAATGTGCCTGTCAAATGGACGAAGCAGGGATTCTGCAAACCCTATGCTACTCCCTCGAGCCGTCAATTGTCTGATTCGTTACCAATTATTTTTTCCTCCTGGATTA"
trim_dict["empty_2"] = 25
wild_type_dict["insulator"] = "ATTCACCACCCTGAATTGACTCTCTTCCGGGCGCTATCATGCCATACCGCGAAAGGTTTTGCGCCATTCGATGGCGCGCCGCCATAAATCTCGGTACCAAATTCCAGAAAAGAGGCCTCCCGAAAGGGGGGCCTTTTTTCGTTTTGGTCCGAGCTGTTGACAATTAATCATCGGCTCGTATAATGTGTGGAATTGTGAGCGGATAACAATTAGCTGTCACCGGATGTGCTTTCCGGTCTGATGAGTCCGTGAGGACGAAACAGCCTCTACAAATAATTTTGTTTAATACTAGCCATCAAGGAGAGCTGCTAC"
trim_dict["insulator"] = 25
wild_type_dict["KAN"] = "AAGCGGGAGACCAGAAACAAAAAAAGGCCCCCCGTTAGGGAGGCCTTCAATAATTGGTTGTGTCTCAAAATCTCTGATGTACATTGCACAAGATAAAAATATATCATCATGAACAATAAAACTGTCTGCTTACATAAACAGTAATACAAGGGGTGTTATGAGCCATATTCAACGGGAAACGTCTTGCTCCAGGCCGCGATTAAATTCCAACATGGATGCTGATTTATATGGGTATAAATGGGCTCGCGATAATGTCGGGCAATCAGGTGCGACAATCTATCGATTGTATGGGAAGCCCGATGCGCCAGAGTTGTTTCTGAAACATGGCAAAGGTAGCGTTGCCAATGATGTTACAGATGAGATGGTCAGACTAAACTGGCTGACGGAATTTATGCCTCTTCCGACCATCAAGCATTTTATCCGTACTCCTGATGATGCATGGTTACTCACCACTGCGATCCCCGGGAAAACAGCATTCCAGGTATTAGAAGAATATCCTGATTCAGGTGAAAATATTGTTGATGCGCTGGCAGTGTTCCTGCGCCGGTTGCATTCGATTCCTGTTTGTAATTGTCCTTTTAACAGCGATCGCGTATTTCGTCTCGCTCAGGCGCAATCACGAATGAATAACGGTTTGGTTGATGCGAGTGATTTTGATGACGAGCGTAATGGCTGGCCTGTTGAACAAGTCTGGAAAGAAATGCATAAGCTTTTGCCATTCTCACCGGATTCAGTCGTCACTCATGGTGATTTCTCACTTGATAACCTTATTTTTGACGAGGGGAAATTAATAGGTTGTATTGATGTTGGACGAGTCGGAATCGCAGACCGATACCAGGATCTTGCCATCCTATGGAACTGCCTCGGTGAGTTTTCTCCTTCATTACAGAAACGGCTTTTTCAAAAATATGGTATTGATAATCCTGATATGAATAAATTGCAGTTTCATTTGATGCTCGATGAGTTTTTCTAAGGTAACTGTCAGACCAAGTTTACTCATATATACTTTAGATTGATTTTTCGTTCCACTGAGCGTCAGACCCC"
trim_dict["KAN"] = 0
wild_type_dict["lacI"] = "TCACTGCCCGCTTTCCAGTCGGGAAACCTGTCGTGCCAGCTGCATTAATGAATCGGCCAACGCGCGGGGAGAGGCGGTTTGCGTATTGGGCGCCAGGGTGGTTTTTCTTTTCACCAGTGAGACTGGCAACAGCTGATTGCCCTTCACCGCCTGGCCCTGAGAGAGTTGCAGCAAGCGGTCCACGCTGGTTTGCCCCAGCAGGCGAAAATCCTGTTTGATGGTGGTTAACGGCGGGATATAACATGAGCTATCTTCGGTATCGTCGTATCCCACTACCGAGATATCCGCACCAACGCGCAGCCCGGACTCGGTAATGGCGCGCATTGCGCCCAGCGCCATCTGATCGTTGGCAACCAGCATCGCAGTGGGAACGATGCCCTCATTCAGCATTTGCATGGTTTGTTGAAAACCGGACATGGCACTCCAGTCGCCTTCCCGTTCCGCTATCGGCTGAATTTGATTGCGAGTGAGATATTTATGCCAGCCAGCCAGACGCAGACGCGCCGAGACAGAACTTAATGGGCCCGCTAACAGCGCGATTTGCTGGTGACCCAATGCGACCAGATGCTCCACGCCCAGTCGCGTACCGTCCTCATGGGAGAAAATAATACTGTTGATGGGTGTCTGGTCAGAGACATCAAGAAATAACGCCGGAACATTAGTGCAGGCAGCTTCCACAGCAATGGCATCCTGGTCATCCAGCGGATAGTTAATGATCAGCCCACTGACGCGTTGCGCGAGAAGATTGTGCACCGCCGCTTTACAGGCTTCGACGCCGCTTCGTTCTACCATCGACACCACCACGCTGGCACCCAGTTGATCGGCGCGAGATTTAATCGCCGCGACAATTTGCGACGGCGCGTGCAGGGCCAGACTGGAGGTGGCAACGCCAATCAGCAACGACTGTTTGCCCGCCAGTTGTTGTGCCACGCGGTTGGGAATGTAATTCAGCTCCGCCATCGCCGCTTCCACTTTTTCCCGCGTTTTCGCAGAAACGTGGCTGGCCTGGTTCACCACGCGGGAAACGGTCTGATAAGAGACACCGGCATACTCTGCGACATCGTATAACGTTACTGGTTTCAT"
trim_dict["lacI"] = 0
wild_type_dict["Ori"] = "TTAATAAGATGATCTTCTTGAGATCGTTTTGGTCTGCGCGTAATCTCTTGCTCTGAAAACGAAAAAACCGCCTTGCAGGGCGGTTTTTCGAAGGTTCTCTGAGCTACCAACTCTTTGAACCGAGGTAACTGGCTTGGAGGAGCGCAGTCACCAAAACTTGTCCTTTCAGTTTAGCCTTAACCGGCGCATGACTTCAAGACTAACTCCTCTAAATCAATTACCAGTGGCTGCTGCCAGTGGTGCTTTTGCATGTCTTTCCGGGTTGGACTCAAGACGATAGTTACCGGATAAGGCGCAGCGGTCGGACTGAACGGGGGGTTCGTGCATACAGTCCAGCTTGGAGCGAACTGCCTACCCGGAACTGAGTGTCAGGCGTGGAATGAGACAAACGCGGCCATAACAGCGGAATGACACCGGTAAACCGAAAGGCAGGAACAGGAGAGCGCACGAGGGAGCCGCCAGGGGGAAACGCCTGGTATCTTTATAGTCCTGTCGGGTTTCGCCACCACTGATTTGAGCGTCAGATTTCGTGATGCTTGTCAGGGGGGCGGAGCCTATGGAAAAACGGCTTTGCCGCGGCCCTCTCACTTCCCTGTTAAGTATCTTCCTGGCATCTTCCAGGAAATCTCCGCCCCGTTCGTAAGCCATTTCCGCTCGCCGCAGTCGAACGACCGAGCGTAGCGAGTCAGTGAGCGAGGAAGCGGAATATATCCTGTATCACATATTCTGCTGACGCACCGGTGCAGCCTTTTTTCTCCTGCCACATGAAGCACTTCACTGACACCCTCATCAGTGCCAACATAGT"
trim_dict["Ori"] = 0
wild_type_dict["tetA"] = "ATGAGTAGCAGTACGAAAATTGCGCTTGTCATCACCCTCCTGGATGCGATGGGGATCGGCTTGATCATGCCGGTACTGCCAACCCTTCTGCGCGAGTTCATTGCAAGCGAAGATATTGCCAACCATTTCGGGGTTCTGCTCGCACTGTACGCCTTAATGCAGGTCATCTTTGCTCCCTGGTTAGGCAAAATGTCAGACAGCTTTGGACGCCGTCCTGTTTTGCTGTTAAGCCTTATCGGAGCGAGCCTGGATTACCTTTTATTGGCCTTCTCCTCGGCACTGTGGATGCTTTATTTGGGTCGTTTGCTGAGTGGGATTACAGGCGCGACGGGTGCCGTGGCGGCGTCGGTGATTGCTGATACGACGTCCGCAAGTCAACGTGTGAAATTGTTCGGCTGGTTAGGAGCCTCCTTTGGCTTGGGCTTAATCGCTGGGCCAATTATTGGCGGGTTCGCCGGCGAAATCTCACCACATTCCCCTTTTTTCATCGCGGCATTACTCAACATTGTCACGTTCCTGGTGGTGATGTTCTGGTTCCGCGAAACGAAAAACACCCGCGATAACACGGATACAGAGGTGGGGGTTGAAACGCAATCGAACAGTGTGTACATCACGCTCTTCAAGACCATGCCCATCCTGCTCATCATCTACTTCTCCGCACAGTTGATTGGGCAAATCCCGGCCACAGTGTGGGTTTTGTTTACGGAAAACCGTTTCGGGTGGAACTCCATGATGGTGGGTTTCTCTCTGGCTGGATTGGGACTTCTGCATAGTGTTTTCCAGGCTTTCGTCGCTGGCCGTATTGCCACAAAGTGGGGAGAAAAAACCGCTGTATTGCTTGGTTTTATCGCAGATAGCTCTGCGTTTGCCTTCTTGGCATTTATTAGCGAAGGCTGGCTCGTGTTTCCGGTATTGATTCTGTTGGCTGGGGGCGGTATCGCATTACCCGCGCTGCAGGGAGTTATGTCTATTCAAACCAAATCACACCAACAAGGAGCGCTGCAAGGCTTACTTGTGTCCCTGACCAACGCAACCGGAGTCATCGGGCCACTTCTGTTCGCTGTAATTTATAACCACTCACTGCCAATTTGGGATGGATGGATCTGGATCATCGGTCTTGCCTTCTACTGCATCATCATTTTGCTGTCAATGACATTCATGCTGACGCCTCAAGCCCAAGGATCTAAACAAGAAACGAGTGCC"
trim_dict["tetA"] = 0
wild_type_dict["YFP"] = "TAACGGCGTAAGGAGGTATTTTTATGGTGTCAAAGGGTGAGGAACTGTTTACGGGGATCGTCCCGATTCTTGTTGAACTTGACGGCGACGTAAATGGTCACAAGTTTTCCGTATCGGGCGAAGGTGAGGGCGATGCGACTTATGGGAAATTAACACTGAAATTCATTTGCACCACCGGAAAACTGCCCGTTCCTTGGCCTACTCTGGTAACCACGTTCGGATATGGTTTACAGTGTTTTGCTCGCTACCCGGACCATATGAAACTGCACGATTTCTTCAAGTCCGCCATGCCGGAGGGCTACGTGCAGGAACGTACAATCTTCTTCAAAGACGATGGTAATTACAAGACCCGTGCTGAAGTTAAATTTGAGGGGGATACTTTAGTCAATCGTATTGAATTGAAGGGGATTGACTTTAAGGAAGACGGTAATATCCTTGGCCACAAGCTTGAATACAACTACAATAGTCACAATGTGTATATTATGGCTGATAAACAGAAGAATGGCATTAAGGTTAACTTTAAGATCCGTCACAATATCGAAGACGGATCTGTCCAGCTTGCTGACCATTATCAGCAAAATACGCCGATCGGTGACGGTCCGGTTTTGTTGCCGGACAACCATTACCTGTCCTATCAGTCGGCGTTAAGTAAAGATCCGAATGAGAAACGCGACCATATGGTTTTGTTGGAGTTTGTGACGGCTGCTGGCATTACGCTTGGGATGGACGAGCTGTATAAATAA"
trim_dict["YFP"] = 0
# primers regions is concatentated sequence from the index primer binding sites
wild_type_dict["primers"] = "CATCGGTGAGCCCGGGCTGT" + "ACGATGCGTCCGGCGTAGAGG"
trim_dict["primers"] = (20, 21)

wild_type_seq = wild_type_dict[region]
adapt_len = trim_dict[region]

In [ ]:
region

In [ ]:
# Dataset no. 1:

In [ ]:
os.chdir(notebook_directory)
os.getcwd()

In [ ]:
pac_bio_dir_1 = notebook_directory[:notebook_directory.find("E-Coli")]
pac_bio_dir_1 += "LacI_CCS_analysis\\engineering-bio-lacI-landscape\\data_1\\processed\\targets"
os.chdir(pac_bio_dir_1)
os.getcwd()

In [ ]:
with gzip.open('barcode_1.tsv.gz', 'rb') as f:
    bc1_frame_1 = pd.read_csv(f, sep="\t", skipinitialspace=True)

with gzip.open('barcode_2.tsv.gz', 'rb') as f:
    bc2_frame_1 = pd.read_csv(f, sep="\t", skipinitialspace=True)
    
region_file = region + '.tsv.gz'
with gzip.open(region_file, 'rb') as f:
    region_frame_1 = pd.read_csv(f, sep="\t", skipinitialspace=True)

In [ ]:
%%time

if region=="primers":
    adapt_len_f = adapt_len[0]
    adapt_len_r = adapt_len[1]
else:
    adapt_len_f = adapt_len
    adapt_len_r = adapt_len
    
if adapt_len_f>0:
    new_seqs = []
    for s in region_frame_1["seq"]:
        up = [ c.isupper() for c in s ]
        if True in up:
            n_s = s[up.index(True)-adapt_len_f:-up[::-1].index(True)+adapt_len_r][:350].upper()
            if region=="primers":
                n_s = n_s[:adapt_len_f] + n_s[-adapt_len_r:]
            new_seqs.append(n_s)
        else:
            new_seqs.append("")
    region_frame_1["seq"] = new_seqs
    region_frame_1 = region_frame_1[region_frame_1["seq"]!=""]
    region_frame_1 = region_frame_1[region_frame_1["seq"].str.len()>=adapt_len_f+adapt_len_r]
region_frame_1 = region_frame_1[~region_frame_1["seq"].isnull()]

In [ ]:
# Dataset no. 2:

In [ ]:
os.chdir(notebook_directory)
os.getcwd()

In [ ]:
pac_bio_dir_2 = notebook_directory[:notebook_directory.find("E-Coli")]
pac_bio_dir_2 += "LacI_CCS_analysis\\engineering-bio-lacI-landscape\\data_2\\processed\\targets"
os.chdir(pac_bio_dir_2)
os.getcwd()

In [ ]:
with gzip.open('barcode_1.tsv.gz', 'rb') as f:
    bc1_frame_2 = pd.read_csv(f, sep="\t", skipinitialspace=True)

with gzip.open('barcode_2.tsv.gz', 'rb') as f:
    bc2_frame_2 = pd.read_csv(f, sep="\t", skipinitialspace=True)
    
region_file = region + '.tsv.gz'
with gzip.open(region_file, 'rb') as f:
    region_frame_2 = pd.read_csv(f, sep="\t", skipinitialspace=True)

In [ ]:
%%time

if region=="primers":
    adapt_len_f = adapt_len[0]
    adapt_len_r = adapt_len[1]
else:
    adapt_len_f = adapt_len
    adapt_len_r = adapt_len
    
if adapt_len_f>0:
    new_seqs = []
    for s in region_frame_2["seq"]:
        up = [ c.isupper() for c in s ]
        if True in up:
            n_s = s[up.index(True)-adapt_len_f:-up[::-1].index(True)+adapt_len_r][:350].upper()
            if region=="primers":
                n_s = n_s[:adapt_len_f] + n_s[-adapt_len_r:]
            new_seqs.append(n_s)
        else:
            new_seqs.append("")
    region_frame_2["seq"] = new_seqs
    region_frame_2 = region_frame_2[region_frame_2["seq"]!=""]
    region_frame_2 = region_frame_2[region_frame_2["seq"].str.len()>=adapt_len_f+adapt_len_r]
region_frame_2 = region_frame_2[~region_frame_2["seq"].isnull()]

In [ ]:
bc1_dict = {}
for key, value in zip(bc1_frame_1["#name"], bc1_frame_1["seq"]):
    bc1_dict[key] = value
for key, value in zip(bc1_frame_2["#name"], bc1_frame_2["seq"]):
    bc1_dict[key] = value
    
bc2_dict = {}
for key, value in zip(bc2_frame_1["#name"], bc2_frame_1["seq"]):
    bc2_dict[key] = value
for key, value in zip(bc2_frame_2["#name"], bc2_frame_2["seq"]):
    bc2_dict[key] = value
    
region_dict = {}
for key, value in zip(region_frame_1["#name"], region_frame_1["seq"]):
    region_dict[key] = value
for key, value in zip(region_frame_2["#name"], region_frame_2["seq"]):
    region_dict[key] = value

In [ ]:
all_keys = np.unique(np.array(list(bc1_dict.keys()) + list(bc2_dict.keys()) + list(region_dict.keys()) ))
len(all_keys)

In [ ]:
%%time
id_list = []
bc2_list = []
bc1_list = []
seq_list = []

for key in all_keys:
    if (key in bc1_dict.keys()) & (key in bc2_dict.keys()) & (key in region_dict.keys()):
        seq = region_dict[key]
        
        if type(seq) is str:
            id_list.append(key[:-3])
            bc1_list.append(bc1_dict[key])
            bc2_list.append(bc2_dict[key])
            seq_list.append(seq)

In [ ]:
print(str(Seq(wild_type_seq).reverse_complement())[:15])
for s in seq_list[:5]:
    print(s[:15])
print(wild_type_seq[:15])

In [ ]:
print(str(Seq(wild_type_seq).reverse_complement())[:15])
for s in seq_list[:5]:
    print(str(Seq(s).reverse_complement())[:15])
print(wild_type_seq[:15])

In [ ]:
seq_list[0][-15:]

In [ ]:
pacbio_frame = pd.DataFrame({"id":id_list, "cterm-bc":bc2_list, "nterm-bc":bc1_list, "seq":seq_list})

In [ ]:
len(pacbio_frame)

In [ ]:
pacbio_frame["seq_length"] = [ len(x) for x in pacbio_frame["seq"] ]

In [ ]:
print(pacbio_frame["seq_length"].mode())

In [ ]:
print(pacbio_frame["seq_length"].max())

In [ ]:
print(pacbio_frame["seq_length"].min())

In [ ]:
seq_length_mode = pacbio_frame["seq_length"].mode().values[0]
seq_length_mode

In [ ]:
len(wild_type_seq)

In [ ]:
plt.rcParams["figure.figsize"] = [8, 6]
fig, axs = plt.subplots(1, 1)
bins= [i+0.5 for i in range(seq_length_mode-10, seq_length_mode+10)]

axs.hist(pacbio_frame["seq_length"], bins=bins, alpha=0.7, label=region_file);
axs.set_yscale('log');
axs.set_xticks([i for i in range(seq_length_mode-10, seq_length_mode+10, 2)]);
leg = axs.legend(loc='upper right', bbox_to_anchor= (0.97, 0.97), ncol=1, borderaxespad=0)
new_length = len(pacbio_frame[pacbio_frame["seq_length"]==seq_length_mode])
print(new_length)
print(new_length/len(pacbio_frame))

In [ ]:
pacbio_frame = pacbio_frame[pacbio_frame["seq_length"]==seq_length_mode].copy()

In [ ]:
pacbio_frame = pacbio_frame[~pacbio_frame["nterm-bc"].isnull()]
pacbio_frame = pacbio_frame[~pacbio_frame["cterm-bc"].isnull()]

In [ ]:
print(len(pacbio_frame))

In [ ]:
%%time

for_match_num = 0
rev_match_num = 0
for_barcodeID_list = []
rev_barcodeID_list = []
for index, row in pacbio_frame.iterrows():
    barcode = row["nterm-bc"]
    #barcode = barcode[1:-2]
    if barcode in for_barcode_clusterID_dict:
        for_barcodeID_list.append(for_barcode_clusterID_dict[barcode])
        for_match_num += 1
    else:
        for_barcodeID_list.append(-1)
    
    barcode = row["cterm-bc"]
    #barcode = reverse_complement(barcode[1:-1])
    barcode = str(Seq(barcode).reverse_complement())
    if barcode in rev_barcode_clusterID_dict:
        rev_barcodeID_list.append(rev_barcode_clusterID_dict[barcode])
        rev_match_num += 1
    else:
        rev_barcodeID_list.append(-1)
        
pacbio_frame["for_BC_ID"] = for_barcodeID_list
pacbio_frame["rev_BC_ID"] = rev_barcodeID_list

print(for_match_num)
print(rev_match_num)

In [ ]:
len(pacbio_frame) - for_match_num

In [ ]:
pacbio_BC_pairs = []

for f_bc, r_bc in zip(pacbio_frame["for_BC_ID"], pacbio_frame["rev_BC_ID"]):
    pacbio_BC_pairs.append(f"{f_bc}_{r_bc}")

pacbio_frame["dual_BC_ID"] = pacbio_BC_pairs

In [ ]:
len(pacbio_frame)

In [ ]:
# fraction of PacBio reads that have one of the matching barcodes
print(len(pacbio_frame[pacbio_frame["for_BC_ID"]!=-1])/len(pacbio_frame))
print(len(pacbio_frame[pacbio_frame["rev_BC_ID"]!=-1])/len(pacbio_frame))

In [ ]:
# fraction of PacBio reads that have both matching barcodes
print(len(pacbio_frame[(pacbio_frame["rev_BC_ID"]>-1) & (pacbio_frame["for_BC_ID"]>-1)])/len(pacbio_frame))

In [ ]:
hiseq_BC_pairs = hiseq_count_frame["dual_BC_ID"]
# fraction of PacBio reads that have dual barcode matching a dual barcode from the HiSeq data
print(len(pacbio_frame[pacbio_frame["dual_BC_ID"].isin(hiseq_BC_pairs)])/len(pacbio_frame))

In [ ]:
len(pacbio_frame[~pacbio_frame["dual_BC_ID"].isin(hiseq_BC_pairs)])

In [ ]:
# fraction of HiSeq double barcodes that have matching barcodes in the PacBio dataset
hiseq_BC_pairs_series = pd.Series(hiseq_BC_pairs)
hiseq_BC_pairs_series_2 = hiseq_BC_pairs_series[hiseq_BC_pairs_series.isin(pacbio_frame["dual_BC_ID"])]
print(len(hiseq_BC_pairs_series_2)/len(hiseq_BC_pairs))

In [ ]:
# number of HiSeq double barcodes that have matching barcodes in the PacBio dataset
print(len(hiseq_BC_pairs_series_2))

In [ ]:
# number of unique dual barcodes found in the PacBio data
#    (not necessarily dual barcode pairs that showed up in the HiSeq)
print(len(pacbio_frame[pacbio_frame["dual_BC_ID"].str.contains("-1")==False]["dual_BC_ID"].unique()))

In [ ]:
#number of barcodes in HiSeq dataset
len(hiseq_count_frame)

In [ ]:
def distance(str1, str2):
    if len(str1) != len(str2):
        raise ValueError("Strand lengths are not equal!")
    else:
        count = 0
        for (a, b) in zip(str1, str2):
            if a!=b:
                if ( (a=='X') or (b=='X') ):
                    count += 0.5
                else:
                    count += 1
                
    return count

In [ ]:
def trim_errors(err_list):
    if len(err_list)<=2:
        return err_list
    else:
        #if there are 3 or more terms, throw out outliers
        err_list.sort()
        #if there are 2 sequences with a low error rate relative to the consensus, assume they are the "good" reads.
        err_list = [err for err in err_list if err<=err_list[1]]
        return err_list

In [ ]:
%%time

#Calculate consensus seq for each dual barcode and seq read error rate relative to consensus

dual_seq_list = list(pacbio_frame[(pacbio_frame["for_BC_ID"]!=-1) & (pacbio_frame["rev_BC_ID"]!=-1)]["seq"].values)

dual_concensus_seq_list = []
dual_seq_err_rate = []
dual_cluster_size_list = []
not_same_length_list = []

for bc_id in hiseq_count_frame["dual_BC_ID"]:
    df = pacbio_frame[pacbio_frame["dual_BC_ID"]==bc_id]
    
    dual_cluster_size_list.append(len(df))
    
    seq_list = df["seq"]
    
    if len(seq_list)>1:
        same_length = True
        for x, y in zip(seq_list[1:], seq_list[:-1]):
            same_length = same_length and (len(x) == len(y))
            
        if same_length:
            alignment = MultipleSeqAlignment([ SeqRecord(Seq(x)) for x in seq_list ])
            summary_align = AlignInfo.SummaryInfo(alignment)
            concensus_seq = str(summary_align.dumb_consensus(threshold=0.2, consensus_alpha=generic_dna))

            dual_concensus_seq_list.append(concensus_seq)
            errors = []
            for c in seq_list:
                errors.append(distance(c, concensus_seq))
            errors = trim_errors(errors)
            dual_seq_err_rate.append(sum(errors))
        else:
            not_same_length_list.append(df)
            dual_concensus_seq_list.append("")
            dual_seq_err_rate.append(0)
    else:
        if len(seq_list)==1:
            dual_concensus_seq_list.append(seq_list.iloc[0])
        else:
            dual_concensus_seq_list.append("")
        dual_seq_err_rate.append(0)
    
hiseq_count_frame["concensus_seq"] = dual_concensus_seq_list
hiseq_count_frame["seq_error_rate"] = dual_seq_err_rate
hiseq_count_frame["pacbio_seq_count"] = dual_cluster_size_list

In [ ]:
len(not_same_length_list)

In [ ]:
len(dual_concensus_seq_list)

In [ ]:
len(hiseq_count_frame)

In [ ]:
plt.rcParams["figure.figsize"] = [8, 6]
fig, axs = plt.subplots(1, 1)
bins= [i-0.5 for i in range(20)]
axs.hist(hiseq_count_frame["pacbio_seq_count"], bins=bins);
axs.set_yscale("log");

In [ ]:
%%time

#Calculate mutation rate of consensus seq relative to wild type seq

concensus_mutation_rate_list = []

hiseq_count_frame["concensus_seq"] = hiseq_count_frame["concensus_seq"].str.replace("X", "N")

for index, row in hiseq_count_frame.iterrows():
    consensus_seq = row["concensus_seq"]
    if consensus_seq!="":
        errors = fitness.hamming_distance(consensus_seq, wild_type_seq, IGNORE_N=True)
        
        concensus_mutation_rate_list.append(errors)
    
    else:
        concensus_mutation_rate_list.append(-1)
    
hiseq_count_frame["concensus_seq_mutations"] = concensus_mutation_rate_list

In [ ]:
plt.rcParams["figure.figsize"] = [8, 6]
fig, axs = plt.subplots(1, 1)
bins= [i-0.5 for i in range(20)]
axs.hist(hiseq_count_frame["concensus_seq_mutations"], bins=bins, label=">0 PacBio Reads");

df = hiseq_count_frame[hiseq_count_frame["pacbio_seq_count"]>10]
axs.hist(df["concensus_seq_mutations"], bins=bins, label=">10 PacBio Reads");
df = hiseq_count_frame[hiseq_count_frame["pacbio_seq_count"]>30]
axs.hist(df["concensus_seq_mutations"], bins=bins, label=">30 PacBio Reads");

axs.set_yscale("log");
axs.set_ylabel("Count")
axs.set_xlabel(region + " Mutations")
leg = axs.legend(loc='upper right', bbox_to_anchor= (0.97, 0.97), ncol=1)

In [ ]:
df = hiseq_count_frame[hiseq_count_frame["pacbio_seq_count"]>2]
df = df[df["concensus_seq_mutations"]>0]
print(len(df))
df = df[df["concensus_seq_mutations"]>1]
print(len(df))

In [ ]:
df = hiseq_count_frame
df = df[df["RS_name"]!=""]
df[["dual_BC_ID", "RS_name", "concensus_seq_mutations", "pacbio_seq_count", "seq_error_rate", "concensus_seq"]]

In [ ]:
df = pacbio_frame[pacbio_frame["dual_BC_ID"]=="31486_25239"]
df[["seq"]]

In [ ]:
pacbio_region_count = hiseq_count_frame["pacbio_seq_count"]
pacbio_region_mutations = hiseq_count_frame["concensus_seq_mutations"]

In [ ]:
# Pause here-

In [ ]:
os.chdir(notebook_directory)
pickle_file = experiment + '_BarSeqFitnessFrame.pkl'
print(pickle_file)

barcode_frame = pickle.load(open(pickle_file, 'rb'))

barcode_frame.barcode_frame["pacbio_" + region + "_count"] = pacbio_region_count
barcode_frame.barcode_frame["pacbio_" + region + "_mutations"] = pacbio_region_mutations

In [ ]:
df = barcode_frame.barcode_frame
warning_list = []
for mut, n in zip(df["pacbio_" + region + "_mutations"], df["pacbio_" + region + "_count"]):
    w = (n>0) and (mut>0)
    warning_list.append(w)
warning_list = np.array(warning_list)
len(warning_list[warning_list])

In [ ]:
len(wild_type_seq)

In [ ]:
barcode_frame.save_as_pickle()

In [ ]:
%%time
# Look at distributions of fitness values - to see if there is an effect from mutations to the sequence region

fitness_0_list = []
fitness_20_list = []
fitness_0_0_list = []
fitness_20_0_list = []
df = barcode_frame.barcode_frame
df = df[df["total_counts"]>3000]

for fit_0_b, fit_20_b in zip(df["fitness_0_estimate_b"], df["fitness_20_estimate_b"]):
    fitness_0_list += list(fit_0_b)
    fitness_0_0_list += list(fit_0_b)[:1]
    fitness_20_list += list(fit_20_b)
    fitness_20_0_list += list(fit_20_b)[:1]
    
fitness_0_list = np.array(fitness_0_list)
fitness_20_list = np.array(fitness_20_list)
fitness_0_0_list = np.array(fitness_0_0_list)
fitness_20_0_list = np.array(fitness_20_0_list)
fitness_diff_list = fitness_20_list - fitness_0_list
normed_diff_list = (fitness_20_list - fitness_0_list)/fitness_0_list
fitness_diff_0_list = fitness_20_0_list - fitness_0_0_list
normed_diff_0_list = (fitness_20_0_list - fitness_0_0_list)/fitness_0_0_list

In [ ]:
%%time
# Look at distributions of fitness values - to see if there is an effect from mutations to the sequence region

fitness_0_region = []
fitness_20_region = []
fitness_0_0_region = []
fitness_20_0_region = []
df = barcode_frame.barcode_frame
df = df[warning_list]
df = df[df["total_counts"]>3000]

for fit_0_b, fit_20_b in zip(df["fitness_0_estimate_b"], df["fitness_20_estimate_b"]):
    fitness_0_region += list(fit_0_b)
    fitness_0_0_region += list(fit_0_b)[:1]
    fitness_20_region += list(fit_20_b)
    fitness_20_0_region += list(fit_20_b)[:1]
    
fitness_0_region = np.array(fitness_0_region)
fitness_20_region = np.array(fitness_20_region)
fitness_0_0_region = np.array(fitness_0_0_region)
fitness_20_0_region = np.array(fitness_20_0_region)
fitness_diff_region = fitness_20_region - fitness_0_region
normed_diff_region = (fitness_20_region - fitness_0_region)/fitness_0_region
fitness_diff_0_region = fitness_20_0_region - fitness_0_0_region
normed_diff_0_region = (fitness_20_0_region - fitness_0_0_region)/fitness_0_0_region

In [ ]:
plt.rcParams["figure.figsize"] = [6, 4]
fig, axs = plt.subplots(1, 1)
bins= [i-0.5 for i in range(20)]
axs.hist(hiseq_count_frame["concensus_seq_mutations"], bins=bins, label=">0 PacBio Reads");

df = hiseq_count_frame[hiseq_count_frame["pacbio_seq_count"]>10]
axs.hist(df["concensus_seq_mutations"], bins=bins, label=">10 PacBio Reads");
df = hiseq_count_frame[hiseq_count_frame["pacbio_seq_count"]>30]
axs.hist(df["concensus_seq_mutations"], bins=bins, label=">30 PacBio Reads");

axs.set_yscale("log");
axs.set_ylabel("Count")
axs.set_xlabel(region + " Mutations")
leg = axs.legend(loc='upper right', bbox_to_anchor= (0.97, 0.97), ncol=1)

# Look at distributions of fitness values - to see if there is an effect from mutations to the sequence region
plt.rcParams["figure.figsize"] = [16, 12]
fig, axs_grid = plt.subplots(3, 3)
fig.suptitle(region + " fitness count ratios", y=0.98, size=24)
axs = axs_grid.flatten()

axs[0].get_shared_y_axes().join(*axs[:3]);

shift = 0.025
for i, (ax_0, ax_1, ax_2) in enumerate(zip(axs[:3], axs[3:6], axs[6:])):
    ax_0.get_shared_x_axes().join(ax_0, ax_1, ax_2);
    for ax in [ax_0, ax_1, ax_2]:
        box = ax.get_position()
        box.x0 = box.x0 + shift*i
        box.x1 = box.x1 + shift*i
        ax.set_position(box)

lists_to_plot = [fitness_0_list, fitness_20_list, normed_diff_list]
lists_to_plot_0 = [fitness_0_0_list, fitness_20_0_list, normed_diff_0_list]
regions_to_plot = [fitness_0_region, fitness_20_region, normed_diff_region]
regions_to_plot_0 = [fitness_0_0_region, fitness_20_0_region, normed_diff_0_region]
names = ["Fitness without Tet", "Fitness with Tet", "Normalized Difference"]

color_list = sns.color_palette()

for ax_0, ax_1, ax_2, data, data_0, reg_data, reg_data_0, name in zip(axs[:3], axs[3:6], axs[6:], lists_to_plot,
                                                                      lists_to_plot_0, regions_to_plot, regions_to_plot_0,
                                                                      names):
    bins = 31
    ax_0.set_ylabel("Count Ratio with Full Library")
    ax_1.set_ylabel("[IPTG]=0 Count")
    ax_2.set_ylabel("All [IPTG] Count")
    alpha=1
    
    if name=="Fitness without Tet":
        bins = np.linspace(0.4,1.3,bins)
    if name=="Fitness with Tet":
        bins = np.linspace(-0.7,1.3,bins)
    if name=="Fitness Difference":
        bins = np.linspace(-1.7,0.5,bins)
    if name=="Normalized Difference":
        bins = np.linspace(-1.7,0.5,bins)
        
    n_all, b, p = ax_2.hist(data, bins=bins, label="Full Library", color=color_list[1])
    n_0, b, p = ax_1.hist(data_0, bins=bins, label="Full Library", color=color_list[0])
    reg_n_all, b, p = ax_2.hist(reg_data, bins=bins, label="Mutated " + region, color="lightgray", alpha=0.85)
    reg_n_0, b, p = ax_1.hist(reg_data_0, bins=bins, label="Mutated " + region, color="lightgray", alpha=0.85)
    
    x = (bins[:-1] + bins[1:])/2
    
    y = reg_n_all/n_all
    yerr = np.sqrt(y/n_all*(1+y))
    ax_0.errorbar(x, y, yerr, fmt="o", alpha=alpha, label="All [IPTG]", color=color_list[1]);
    
    y = reg_n_0/n_0
    yerr = np.sqrt(y/n_0*(1+y))
    ax_0.errorbar(x, y, yerr, fmt="o", alpha=alpha, label="[IPTG] = 0", color=color_list[0]);
    ax_0.set_yscale("log");
    ax_1.set_yscale("log");
    ax_2.set_yscale("log");
    ax_2.set_xlabel(name)
leg = axs[0].legend(loc='lower left', bbox_to_anchor= (0.03, 1.03), ncol=1, borderaxespad=0, frameon=True, fontsize=12)
leg.get_frame().set_edgecolor('k');
for ax in [axs[3], axs[6]]:
    leg = ax.legend(loc='upper left', bbox_to_anchor= (0.03, 0.97), ncol=1, borderaxespad=0, frameon=True, fontsize=12)
    leg.get_frame().set_edgecolor('k');

df = barcode_frame.barcode_frame
df_w = df[warning_list]
df = df[df["total_counts"]>3000]
df_w = df_w[df_w["total_counts"]>3000]
total_ratio = len(df_w)/len(df)
axs[0].set_ylim(total_ratio/10, total_ratio*10)
for ax in axs[:3]:
    xlim = ax.get_xlim()
    ax.plot(xlim, [total_ratio]*2, "--k")
    ax.set_xlim(xlim)

In [ ]:
region

In [ ]:
# Look at distributuion and location of mutations - to see if there are any paterns

# For each variant, create list of DNA changes from wild-type sequence
df = barcode_frame.barcode_frame
dna_list = list(hiseq_count_frame["concensus_seq"])
dna_distance = list(df["pacbio_" + region + "_mutations"])

mutations_lists = []
position_lists = []

for seq, dist in zip(dna_list, dna_distance):
    mutations = []
    positions = []
    if (dist>0):
        for ind, (c1, c2) in enumerate(zip(seq, wild_type_seq)): 
            if (c1 != c2) and (c1 != 'N'):
                mutations.append(f"{c2}{ind+1}{c1}")
                positions.append(ind+1)
    mutations_lists.append(mutations)
    position_lists.append(positions)
    
hiseq_count_frame[region + "_mutation_codes"] = mutations_lists
hiseq_count_frame[region + "_mutation_positions"] = position_lists

In [ ]:
df = hiseq_count_frame
df = df[df["concensus_seq_mutations"]>=1]
df = df[df["concensus_seq_mutations"]<5]

df_rand = hiseq_count_frame.sample(len(df))

for param, lab in zip(["Low Level", "High Level", "IC50"], ["$G_0$", "$G_{∞}$", "$EC_{50}$"]):
    log_low = df[param]
    log_low_rand = df_rand[param]
    positions = df[region + "_mutation_positions"]

    x_list = []
    y_list = []
    y_rand_list = []

    for y, p_list, y_rand in zip(log_low, positions, log_low_rand):
        for p in p_list:
            x_list.append(p)
            y_list.append(y)
            y_rand_list.append(y_rand)
    x_list = np.array(x_list)
    y_list = np.array(y_list)
    y_rand_list = np.array(y_rand_list)
    x_av = np.unique(x_list)
    y_av = [np.mean(y_list[x_list==x]) for x in x_av]
    y_r_av = [np.mean(y_rand_list[x_list==x]) for x in x_av]

    plt.rcParams["figure.figsize"] = [16,5]
    fig, axs = plt.subplots(1, 2)
    axs[0].get_shared_y_axes().join(*axs);

    axs[0].plot(x_list, y_list, "o", alpha=0.2);
    axs[0].set_title(f"variants with {region} mutations", size=20)
    axs[1].plot(x_list, y_rand_list, "o", alpha=0.2);
    axs[1].set_title("randomly selected variants", size=20)
    for ax in axs:
        ax.set_yscale("log");
        ax.set_xlabel(f"Position of Mutation in {region} sequence")
        ax.set_ylabel(lab)

In [ ]:
len(wild_type_seq)

In [ ]:
df = hiseq_count_frame
df = df[df["concensus_seq_mutations"]>=1]
df = df[df["concensus_seq_mutations"]<5]

df_rand = hiseq_count_frame.sample(len(df))

fitness = [x[0] for x in df["fitness_20_estimate_b"] ]
fitness_rand = [x[0] for x in df_rand["fitness_20_estimate_b"] ]
positions = df[region + "_mutation_positions"]

x_list = []
y_list = []
y_rand_list = []

for y, p_list, y_rand in zip(fitness, positions, fitness_rand):
    for p in p_list:
        x_list.append(p)
        y_list.append(y)
        y_rand_list.append(y_rand)
x_list = np.array(x_list)
y_list = np.array(y_list)
y_rand_list = np.array(y_rand_list)
x_av = np.unique(x_list)
y_av = [np.mean(y_list[x_list==x]) for x in x_av]
y_r_av = [np.mean(y_rand_list[x_list==x]) for x in x_av]
        
plt.rcParams["figure.figsize"] = [16,5]
fig, axs = plt.subplots(1, 2)

axs[0].plot(x_list, y_list, "o", alpha=0.3);
axs[0].set_title(f"variants with {region} mutations", size=20)
print(np.mean(y_list))
axs[1].plot(x_list, y_rand_list, "o", alpha=0.3);
axs[1].set_title("randomly selected variants", size=20)
print(np.mean(y_rand_list))
for ax in axs:
    #ax.set_yscale("log");
    ax.set_xlabel(f"Position of Mutation in {region} sequence")
    ax.set_ylabel("Fitness with tet and [IPTG]=0")
ylim = axs[0].get_ylim()
axs[1].set_ylim(ylim);

In [ ]:
df = hiseq_count_frame
df = df[df["concensus_seq_mutations"]>=1]
df = df[df["concensus_seq_mutations"]<5]

df_rand = hiseq_count_frame.sample(len(df))

fitness = [x[-1] for x in df["fitness_20_estimate_b"] ]
fitness_rand = [x[-1] for x in df_rand["fitness_20_estimate_b"] ]
positions = df[region + "_mutation_positions"]

x_list = []
y_list = []
y_rand_list = []

for y, p_list, y_rand in zip(fitness, positions, fitness_rand):
    for p in p_list:
        x_list.append(p)
        y_list.append(y)
        y_rand_list.append(y_rand)
x_list = np.array(x_list)
y_list = np.array(y_list)
y_rand_list = np.array(y_rand_list)
x_av = np.unique(x_list)
y_av = [np.mean(y_list[x_list==x]) for x in x_av]
y_r_av = [np.mean(y_rand_list[x_list==x]) for x in x_av]
        
plt.rcParams["figure.figsize"] = [16,5]
fig, axs = plt.subplots(1, 2)

axs[0].plot(x_list, y_list, "o", alpha=0.3);
axs[0].set_title(f"variants with {region} mutations", size=20)
print(np.mean(y_list))
axs[1].plot(x_list, y_rand_list, "o", alpha=0.3);
axs[1].set_title("randomly selected variants", size=20)
print(np.mean(y_rand_list))
for ax in axs:
    #ax.set_yscale("log");
    ax.set_xlabel(f"Position of Mutation in {region} sequence")
    ax.set_ylabel("Fitness with tet and [IPTG]=2048")
ylim = axs[0].get_ylim()
axs[1].set_ylim(ylim);

In [ ]:
df = hiseq_count_frame
df = df[df["concensus_seq_mutations"]>=1]
df = df[df["concensus_seq_mutations"]<5]

df_rand = hiseq_count_frame.sample(len(df))

fitness = [x[0] for x in df["fitness_0_estimate_b"] ]
fitness_rand = [x[0] for x in df_rand["fitness_0_estimate_b"] ]
positions = df[region + "_mutation_positions"]

x_list = []
y_list = []
y_rand_list = []

for y, p_list, y_rand in zip(fitness, positions, fitness_rand):
    for p in p_list:
        x_list.append(p)
        y_list.append(y)
        y_rand_list.append(y_rand)
x_list = np.array(x_list)
y_list = np.array(y_list)
y_rand_list = np.array(y_rand_list)
x_av = np.unique(x_list)
y_av = [np.mean(y_list[x_list==x]) for x in x_av]
y_r_av = [np.mean(y_rand_list[x_list==x]) for x in x_av]
        
plt.rcParams["figure.figsize"] = [16,5]
fig, axs = plt.subplots(1, 2)

axs[0].plot(x_list, y_list, "o", alpha=0.2);
axs[0].set_title(f"variants with {region} mutations", size=20)
print(np.mean(y_list))
axs[1].plot(x_list, y_rand_list, "o", alpha=0.2);
axs[1].set_title("randomly selected variants", size=20)
print(np.mean(y_rand_list))
for ax in axs:
    #ax.set_yscale("log");
    ax.set_xlabel(f"Position of Mutation in {region} sequence")
    ax.set_ylabel("Fitness without tet and [IPTG]=0")
ylim = axs[0].get_ylim()
axs[1].set_ylim(ylim);

In [ ]:
out_frame = hiseq_count_frame[["dual_BC_ID", "pacbio_seq_count", "concensus_seq", "concensus_seq_mutations",
                               region + "_mutation_codes", region + "_mutation_positions"]].copy()

out_frame.rename(columns={"concensus_seq": "concensus_"+region+"_seq", "concensus_seq_mutations": region+"_mutations"},
                 inplace=True)

out_file = region + "_pacbio_frame.pkl"

with open(out_file, 'wb') as f:
    pickle.dump(out_frame, f)